In [15]:
import numpy as np
from typing import *

# Рассказываю, как работает вообще тема вся эта с вычислением уравнения
## Лайк за красоту решения!

Во-первых, обозначим аргументы - у нас есть:
1. интеграл определенный на области [a, b]
2. вырожденное ядро K(x, y) - здесь kernel_func
3. некоторая известная функция возмущения f(x) - здесь func или f
4. некоторая неизвестная функция u(x) - здесь никак не обозначена
5. некоторый параметр lambda - без понятия как в md символы особые вставлять

___
Все это добро предстает в виде интегральной функции, которое необходимо перевести в дискретный вид, в котором в свою очередь аргументы считаются по области Xi = a + i * h + h / 2, где h = |b - a| / N, где N - число разбиение отрезка [a, b]
___
Дальнейшие преобразования позволяют представить все выражение в матричной форме (I + K) * u = f,
откуда как раз можно сделать замену (I + K) = A и получить стандартное матричное уравнение Au = f

## Инициализируем значения на отрезке

Функция принимает N - число разбиений, a - левую границу и b - правую границу отрезка

На выходе получется кортеж с двумя параметрами:
1. значения X по числу разбиений
2. знаение шага разбиения h

In [16]:
def init_x_h(N: float, a: float, b: float) -> tuple[Sequence[float], float]:
    h = abs(b - a) / N
    x = [i * h  + h / 2 + a for i in range(N)]
    return x, h

## Инициализация аргумента (I + K)

Поскольку вырожденное ядро - функция, то на некотором определенном наборе X можно будет посчитать значения функции в конкретных точках

Затем можно будет добавить значение единичной матрицы и считать аргумент готовым к исполнению

Функция принимает в себя уже известные арргументы N, a, b, x и h, а также еще и функцию6 представляющую ядро - kernel_func

Возвращает матрицу значений **(I + K)**, которую мы будем называть **A** в дальнейшем

In [17]:
def init_a(
            N: float,
            a: float,
            b: float,
            kernel_func: Callable[[float, float], float],
            x: Sequence[float],
            h: float) -> np.matrix:
    kernel_func_h = lambda x, y: kernel_func(x, y) * h
    kernel = [ [kernel_func_h(i, j) for j in x] for i in x ]
    return np.identity(N) + np.matrix(kernel)


## Функция инициализации второго аргумента, в нашем случае вектора f функции возмущения

По аналогии с ядром выичисляется на наборе X

Приниамет следущие аргументы:
1. функцию вычисления f - func
2. вектор X

Возвращает вектор типа np.array вычисленных значений f

In [18]:
def init_b(func: Callable[[float], float], x: Sequence[float]) -> np.array:
    return np.array([func(i) for i in x])

## Функция создания вычисляемого выражения

Принимает в себя все определенные аргументы изначального уравнения:
1. число разбиений - N
2. левую границу участка - a
3. правую границу участка - b
4. функцию ядра - kernel_func
5. функцию внешнего возмущения - func

Возвращает функцию, не принимающую аргументов, вызов которой запускает процесс вычисления, основанный на np.linalg.solve

In [19]:
def init_equation(
            N: float,
            a: float,
            b: float,
            kernel_func: Callable[[float, float], float],
            func: Callable[[float], float]) -> Callable[[], np.array]:
    x, h = init_x_h(N, a, b)
    A = init_a(N, a, b, kernel_func, x, h)
    B = init_b(func, x)
    return lambda: np.linalg.solve(A, B)

In [20]:
a = 0
b = 1
N = 3
kernel_func = lambda x, y: x * y
func = lambda x: x

In [21]:
equation = init_equation(N, a, b, kernel_func, func)
equation()

array([0.12587413, 0.37762238, 0.62937063])